<a href="https://colab.research.google.com/github/Danusyasundaravel/vehicle-detection-and-attendance-system/blob/main/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from PIL import Image
import os

folder_path = "/content/drive/MyDrive/Vehicle"

for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        try:
            img = Image.open(file_path)
            img.verify()  # Verify that it is an image
        except (IOError, SyntaxError) as e:
            print(f"Corrupted file: {file_path}")
            os.remove(file_path)  # Remove corrupted file


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import os
# Step 1: Define paths and parameters
train_data_dir = "/content/drive/MyDrive/Vehicle"  # Replace with the path to your dataset
image_size = (224, 224)  # Image size suitable for MobileNetV2
batch_size = 32
classes = ["Mahindra Scorpio", "Truck", "non-vechicles","scanned vehicle image"]

# Step 2: Data augmentation and loading
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% validation split
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)



Found 1588 images belonging to 4 classes.
Found 395 images belonging to 4 classes.


In [ ]:
# Step 3: Load the pretrained model (MobileNetV2)
base_model = MobileNetV2(
    input_shape=(image_size[0], image_size[1], 3),
    include_top=False,  # Remove the fully connected layer
    weights="imagenet"  # Use weights pre-trained on ImageNet
)

# Freeze the base model layers
base_model.trainable = False

# Step 4: Build the Transfer Learning Model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(len(classes), activation="softmax")  # Output layer for your classes
])

# Step 5: Compile the Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Step 6: Train the Model
epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)



9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/10
50/50 [==============================] - 169s 3s/step - loss: 0.3925 - accuracy: 0.8501 - val_loss: 0.1395 - val_accuracy: 0.9494
Epoch 2/10
50/50 [==============================] - 157s 3s/step - loss: 0.1686 - accuracy: 0.9339 - val_loss: 0.1013 - val_accuracy: 0.9620
Epoch 3/10
50/50 [==============================] - 160s 3s/step - loss: 0.1112 - accuracy: 0.9591 - val_loss: 0.0960 - val_accuracy: 0.9570
Epoch 4/10
50/50 [==============================] - 158s 3s/step - loss: 0.0884 - accuracy: 0.9691 - val_loss: 0.0757 - val_accuracy: 0.9646
Epoch 5/10
50/50 [==============================] - 157s 3s/step - loss: 0.0765 - accuracy: 0.9736 - val_loss: 0.1216 - val_accuracy: 0.9468
Epoch 6/10
50/50 [==============================] - 159s 3s/step - loss: 0.0664 - accuracy: 0.9754 - val_loss: 0.0656 - val_accuracy: 0.9696
Epoch 7/10
50/50 [==============================] - 156s 3s/step - loss: 0.0954 - accuracy:

In [ ]:
model.save("vehicle_detection_model.h5")
model.save("vehicle_detection.keras")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model
model = load_model("/content/vehicle_detection_model.h5")

In [ ]:
# Step 7: Fine-tune the Model (optional)
# Unfreeze the base model and train with a lower learning rate
base_model.trainable = True
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
fine_tune_epochs = 5
history_fine_tune = model.fit(
    train_generator,
    epochs=fine_tune_epochs,
    validation_data=validation_generator
)



Epoch 1/5
50/50 [==============================] - 418s 8s/step - loss: 0.2707 - accuracy: 0.9074 - val_loss: 0.1403 - val_accuracy: 0.9494
Epoch 2/5
50/50 [==============================] - 396s 8s/step - loss: 0.0539 - accuracy: 0.9849 - val_loss: 0.0618 - val_accuracy: 0.9823
Epoch 3/5
50/50 [==============================] - 389s 8s/step - loss: 0.0353 - accuracy: 0.9906 - val_loss: 0.0887 - val_accuracy: 0.9772
Epoch 4/5
50/50 [==============================] - 388s 8s/step - loss: 0.0357 - accuracy: 0.9893 - val_loss: 0.1037 - val_accuracy: 0.9671
Epoch 5/5
50/50 [==============================] - 386s 8s/step - loss: 0.0224 - accuracy: 0.9912 - val_loss: 0.0625 - val_accuracy: 0.9772


In [ ]:
model.save("detection_model1.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Step 8: Function to classify a new image
def classify_image(image_path):
    img = load_img(image_path, target_size=image_size)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    predictions = model.predict(img_array)
    predicted_class = classes[np.argmax(predictions)]
    confidence = np.max(predictions) * 100
    return predicted_class, confidence

# Test the model
image_path = "/content/Datacluster Truck (92).jpg"  # Replace with a test image path
predicted_class, confidence = classify_image(image_path)
print(f"Predicted class: {predicted_class} with confidence: {confidence:.2f}%")


1/1 [==============================] - 1s 766ms/step
Predicted class: Truck with confidence: 99.33%


In [ ]:
# Step 6: Function to classify a new image
def classify_image(image_path):
    img = load_img(image_path, target_size=image_size)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    predictions = model.predict(img_array)
    predicted_class = classes[np.argmax(predictions)]
    confidence = np.max(predictions) * 100
    return predicted_class, confidence

# Test the model
image_path = "/content/68.jpg"  # Replace with a test image path
predicted_class, confidence = classify_image(image_path)
print(f"Predicted class: {predicted_class} with confidence: {confidence:.2f}%")


1/1 [==============================] - 0s 44ms/step
Predicted class: Mahindra Scorpio with confidence: 99.07%


In [ ]:
# Step 6: Function to classify a new image
def classify_image(image_path):
    img = load_img(image_path, target_size=image_size)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    predictions = model.predict(img_array)
    predicted_class = classes[np.argmax(predictions)]
    confidence = np.max(predictions) * 100
    return predicted_class, confidence

# Test the model
image_path = "/content/IMG_20241203_222215.jpg"  # Replace with a test image path
predicted_class, confidence = classify_image(image_path)
print(f"Predicted class: {predicted_class} with confidence: {confidence:.2f}%")


1/1 [==============================] - 0s 44ms/step
Predicted class: scanned vehicle image with confidence: 100.00%


In [ ]:
# Step 6: Function to classify a new image
def classify_image(image_path):
    img = load_img(image_path, target_size=image_size)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    predictions = model.predict(img_array)
    predicted_class = classes[np.argmax(predictions)]
    confidence = np.max(predictions) * 100
    return predicted_class, confidence

# Test the model
image_path = "/content/image1838.png"  # Replace with a test image path
predicted_class, confidence = classify_image(image_path)
print(f"Predicted class: {predicted_class} with confidence: {confidence:.2f}%")


1/1 [==============================] - 0s 45ms/step
Predicted class: non-vechicles with confidence: 100.00%
